# Natural Language Processing and Text Analytics Final Exam [KAN-CDSCO1002U]

Afnan El-Segaier (137863)  
Une Aspelin (152294)  
Kristin Sundby (167303)  
Marlin Haavengen (167342)



---



This notebook outlines the set-up of a **LLaMa Model** for quesion-answering, involving the following four steps:


1.   Preparation and installment of necessary packages.
2.   Sign-in to Huggingface (may require account creation).
3.   Creation of LLaMA "text-generation" pipeline using Huggingface transformers.
4.   Creation of a get_llama_response function.

## SVM Model

###**Step 1: Install Necessary Packages**


In [ ]:
!pip install transformers torch accelerate
import torch
from transformers import AutoTokenizer
from transformers import pipeline

###**Step 2: Sign in to Huggingface**

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).

In [ ]:
# Verifying successful log-in:
!huggingface-cli whoami

kristinsundby


###**Step 3: Creating LLaMA "text-generation" Pipeline**

In [ ]:
# Setting the random seed for PyTorch to 42, to ensure reproducibility:
torch.manual_seed(42)

# Defining model name for the LLaMA:
model = "meta-llama/Llama-2-7b-chat-hf"

# Initializing the LLaMA tokenizer using the AutoTokenizer class from Hugging Face:
tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True) # The use_auth_token=True parameter indicates that an authentication token is required to access the model.

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Using the pipeline function from Hugging Face to create a LLaMA pipeline for text generation:
llama_pipeline = pipeline(
    "text-generation", # Specifying text generation as the task.
    model=model, # Using the LLaMA model specified earlier.
    torch_dtype=torch.float16, # Optional - torch data type specified as float16 for reduced memory usage.
    device_map="auto", # This automatically assign the model to the available device (e.g., GPU or CPU).
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

###**Step 4: Creating The 'get_llama_response' Function:**

In [ ]:
# The function takes user input as a prompt to generate a response from the LLaMA model, and returns the model's response:
def get_llama_response(prompt: str) -> None:

    # Using the LLaMA pipeline to generate sequences based on the provided prompt.
    sequences = llama_pipeline(
        prompt, # The prompt text.
        do_sample=True, # Enables sampling to generate diverse responses.
        top_k=10, # Limiting the sampling to the top 10 tokens with highest probabilities.
        num_return_sequences=1, # Generate only one sequence.
        eos_token_id=tokenizer.eos_token_id, # The end-of-sequence token ID.
        max_length=256, # Maximum length of the generated sequence limited to 256.
    )
    # Print the generated response:
    print("Chatbot:", sequences[0]['generated_text'])

###**Step 5: Pose Questions For Sarcasm Detection**


**Zero Shot Prompting:**

In [ ]:
prompt_1 = '''Classify the sentence as sarcastic or non-sarcastic.
Sentence: Biden is a great President like none other we have had'''
get_llama_response(prompt_1)

Chatbot: Classify the sentence as sarcastic or non-sarcastic.
Sentence: Biden is a great President like none other we have had.
Classification: Sarcastic
Explanation: The sentence is sarcastic because it is an exaggerated and unlikely statement about President Biden, who has been criticized by many for his political actions and decisions. The use of the word "great" in this context is meant to convey the opposite meaning, implying that Biden is not actually a great President.


In [ ]:
prompt_2 = '''Classify the sentence as sarcastic or non-sarcastic.
Sentence: Max Verstappen is such a clean driver, he never makes dirty moves when racing.'''
get_llama_response(prompt_2)

Chatbot: Classify the sentence as sarcastic or non-sarcastic.
Sentence: Max Verstappen is such a clean driver, he never makes dirty moves when racing.
Classification: Sarcastic
Explanation: This sentence is sarcastic because it is a clear exaggeration of Max Verstappen's driving style. The statement implies that he is always clean and never makes dirty moves, which is unlikely and contradicts the driver's reputation for being aggressive and controversial on the track.


In [ ]:
prompt_3 = '''Classify the sentence as sarcastic or non-sarcastic.
Sentence: I just can’t wait to spend time with my family over Christmas! I just love being the only single one and the many many questions asking when will I get a boyfriend 🙄'''
get_llama_response(prompt_3)

Chatbot: Classify the sentence as sarcastic or non-sarcastic.
Sentence: I just can’t wait to spend time with my family over Christmas! I just love being the only single one and the many many questions asking when will I get a boyfriend 🙄.

Classification: Sarcastic

Explanation: The sentence is sarcastic because it exaggerates the annoyance of being asked when the speaker will get a boyfriend, implying that it is a constant and unwanted topic of conversation. The use of the eye-rolling emoji 🙄 also suggests sarcasm.


In [ ]:
prompt_4 = '''Classify the sentence as sarcastic or non-sarcastic.
Sentence: Well this is awesome news to wake up to! '''
get_llama_response(prompt_4)

Chatbot: Classify the sentence as sarcastic or non-sarcastic.
Sentence: Well this is awesome news to wake up to! 😅

(Note: The emojis are used to convey the tone of the sentence)

Classification: Sarcastic


In [ ]:
prompt_5 = '''Classify the sentence as sarcastic or non-sarcastic.
Sentence: So school have sent out a really helpful newsletter with all key dates and activities. So many fun things arranged for the children.'''
get_llama_response(prompt_5)

Chatbot: Classify the sentence as sarcastic or non-sarcastic.
Sentence: So school have sent out a really helpful newsletter with all key dates and activities. So many fun things arranged for the children. (Sarcastic)

Answer: Sarcastic

Explanation: The speaker is using sarcasm to express their disbelief or annoyance at the amount of activities and dates listed in the newsletter. The use of the word "so" and the exaggerated language ("so many fun things arranged for the children") are both indicative of sarcasm.


In [ ]:
prompt_6 = '''Classify the sentence as sarcastic or non-sarcastic.
Sentence: I just love the smell of one million 😍'''
get_llama_response(prompt_6)

Chatbot: Classify the sentence as sarcastic or non-sarcastic.
Sentence: I just love the smell of one million 😍
Classification: Sarcastic
Explanation: The use of the emoticon 😍 and the number one million are both exaggerated and hyperbolic, indicating that the speaker does not actually have a strong positive emotion towards the thing being described.


**Few Shot Prompting:**

In [ ]:
prompt_1 = '''
Sarcasm is the use of irony to mock or convey contempt. Here are some examples:
Sarcastic: “I love waking up to start work at 5am everyday. Even better when its Christmas, because there's nothing I love more than being surrounded by stressed out customers over food!”
Sarcastic: “Love it when the milk is out of date when you buy it”
Non-sarcastic: “I simply love this clip. Thank you.”

Given these examples, classify the following sentence as sarcastic or non-sarcastic: "Biden is a great President like none other we have had"
'''
get_llama_response(prompt_1)

Chatbot: 
Sarcasm is the use of irony to mock or convey contempt. Here are some examples:
Sarcastic: “I love waking up to start work at 5am everyday. Even better when its Christmas, because there's nothing I love more than being surrounded by stressed out customers over food!”
Sarcastic: “Love it when the milk is out of date when you buy it”
Non-sarcastic: “I simply love this clip. Thank you.”

Given these examples, classify the following sentence as sarcastic or non-sarcastic: "Biden is a great President like none other we have had"

Answer: Non-sarcastic. The sentence is expressing genuine admiration and approval for President Biden, without any hint of irony or mockery.


In [ ]:
prompt_2 = '''
Sarcasm is the use of irony to mock or convey contempt. Here are some examples:
Sarcastic: “I love waking up to start work at 5am everyday. Even better when its Christmas, because there's nothing I love more than being surrounded by stressed out customers over food!”
Sarcastic: “Love it when the milk is out of date when you buy it”
Non-sarcastic: “I simply love this clip. Thank you.”

Given these examples, classify the following sentence as sarcastic or non-sarcastic: "Max Verstappen is such a clean driver, he never makes dirty moves when racing."
'''
get_llama_response(prompt_2)

Chatbot: 
Sarcasm is the use of irony to mock or convey contempt. Here are some examples:
Sarcastic: “I love waking up to start work at 5am everyday. Even better when its Christmas, because there's nothing I love more than being surrounded by stressed out customers over food!”
Sarcastic: “Love it when the milk is out of date when you buy it”
Non-sarcastic: “I simply love this clip. Thank you.”

Given these examples, classify the following sentence as sarcastic or non-sarcastic: "Max Verstappen is such a clean driver, he never makes dirty moves when racing."

Answer: Non-sarcastic. The sentence is a genuine statement of admiration for Max Verstappen's driving skills, and does not contain any irony or mocking tone that is characteristic of sarcasm.


In [ ]:
prompt_3 = '''
Sarcasm is the use of irony to mock or convey contempt. Here are some examples:
Sarcastic: “I love waking up to start work at 5am everyday. Even better when its Christmas, because there's nothing I love more than being surrounded by stressed out customers over food!”
Sarcastic: “Love it when the milk is out of date when you buy it”
Non-sarcastic: “I simply love this clip. Thank you.”

Given these examples, classify the following sentence as sarcastic or non-sarcastic: "I just can’t wait to spend time with my family over Christmas! I just love being the only single one and the many many questions asking when will I get a boyfriend 🙄"
'''
get_llama_response(prompt_3)

Chatbot: 
Sarcasm is the use of irony to mock or convey contempt. Here are some examples:
Sarcastic: “I love waking up to start work at 5am everyday. Even better when its Christmas, because there's nothing I love more than being surrounded by stressed out customers over food!”
Sarcastic: “Love it when the milk is out of date when you buy it”
Non-sarcastic: “I simply love this clip. Thank you.”

Given these examples, classify the following sentence as sarcastic or non-sarcastic: "I just can’t wait to spend time with my family over Christmas! I just love being the only single one and the many many questions asking when will I get a boyfriend 🙄"

Answer: Sarcastic


In [ ]:
prompt_4 = '''
Sarcasm is the use of irony to mock or convey contempt. Here are some examples:
Sarcastic: “I love waking up to start work at 5am everyday. Even better when its Christmas, because there's nothing I love more than being surrounded by stressed out customers over food!”
Sarcastic: “Love it when the milk is out of date when you buy it”
Non-sarcastic: “I simply love this clip. Thank you.”

Given these examples, classify the following sentence as sarcastic or non-sarcastic: "Well this is awesome news to wake up to!"
'''
get_llama_response(prompt_4)

Chatbot: 
Sarcasm is the use of irony to mock or convey contempt. Here are some examples:
Sarcastic: “I love waking up to start work at 5am everyday. Even better when its Christmas, because there's nothing I love more than being surrounded by stressed out customers over food!”
Sarcastic: “Love it when the milk is out of date when you buy it”
Non-sarcastic: “I simply love this clip. Thank you.”

Given these examples, classify the following sentence as sarcastic or non-sarcastic: "Well this is awesome news to wake up to!"

Answer: Sarcastic


In [ ]:
prompt_5 = '''
Sarcasm is the use of irony to mock or convey contempt. Here are some examples:
Sarcastic: “I love waking up to start work at 5am everyday. Even better when its Christmas, because there's nothing I love more than being surrounded by stressed out customers over food!”
Sarcastic: “Love it when the milk is out of date when you buy it”
Non-sarcastic: “I simply love this clip. Thank you.”

Given these examples, classify the following sentence as sarcastic or non-sarcastic: "So school have sent out a really helpful newsletter with all key dates and activities. So many fun things arranged for the children."
'''
get_llama_response(prompt_5)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Chatbot: 
Sarcasm is the use of irony to mock or convey contempt. Here are some examples:
Sarcastic: “I love waking up to start work at 5am everyday. Even better when its Christmas, because there's nothing I love more than being surrounded by stressed out customers over food!”
Sarcastic: “Love it when the milk is out of date when you buy it”
Non-sarcastic: “I simply love this clip. Thank you.”

Given these examples, classify the following sentence as sarcastic or non-sarcastic: "So school have sent out a really helpful newsletter with all key dates and activities. So many fun things arranged for the children."

Answer: Non-sarcastic. The speaker is genuinely expressing gratitude and enthusiasm for the newsletter, without any hint of irony or mocking tone.


In [ ]:
prompt_6 = '''
Sarcasm is the use of irony to mock or convey contempt. Here are some examples:
Sarcastic: “I love waking up to start work at 5am everyday. Even better when its Christmas, because there's nothing I love more than being surrounded by stressed out customers over food!”
Sarcastic: “Love it when the milk is out of date when you buy it”
Non-sarcastic: “I simply love this clip. Thank you.”

Given these examples, classify the following sentence as sarcastic or non-sarcastic: "I just love the smell of one million 😍"
'''
get_llama_response(prompt_6)

Chatbot: 
Sarcasm is the use of irony to mock or convey contempt. Here are some examples:
Sarcastic: “I love waking up to start work at 5am everyday. Even better when its Christmas, because there's nothing I love more than being surrounded by stressed out customers over food!”
Sarcastic: “Love it when the milk is out of date when you buy it”
Non-sarcastic: “I simply love this clip. Thank you.”

Given these examples, classify the following sentence as sarcastic or non-sarcastic: "I just love the smell of one million 😍"

Sarcastic. The use of the word "love" in this sentence is ironic, as the speaker clearly does not actually love the smell of one million 😍.


**Chain of Thought:**

In [ ]:
prompt_1 = '''
Q: Classify the following sentence as sarcastic or non-sarcastic: "Love it when the milk is out of date when you buy it"
A: Milk is not supposed to be out of date when you buy it. Buying expired milk is an annoying experience. The sentence expresses the opposite of what the writer feels in the situation. Therefore, the sentence is sarcastic.

Q: Classify the following sentence as sarcastic or non-sarcastic: “Biden is a great President like none other we have had”
'''
get_llama_response(prompt_1)

Chatbot: 
Q: Classify the following sentence as sarcastic or non-sarcastic: "Love it when the milk is out of date when you buy it"
A: Milk is not supposed to be out of date when you buy it. Buying expired milk is an annoying experience. The sentence expresses the opposite of what the writer feels in the situation. Therefore, the sentence is sarcastic.

Q: Classify the following sentence as sarcastic or non-sarcastic: “Biden is a great President like none other we have had”
A: This sentence is non-sarcastic. It is a genuine expression of admiration for President Biden. The use of the word "great" and "none other" emphasizes the speaker's positive opinion of Biden.

Q: Classify the following sentence as sarcastic or non-sarcastic: "Wow, what a surprise, the traffic is terrible today"
A: This sentence is sarcastic. The use of the word "wow" and "what a surprise" is an ironic way of expressing the speaker's frustration with the traffic.


In [ ]:
prompt_2 = '''
Q: Classify the following sentence as sarcastic or non-sarcastic: "Love it when the milk is out of date when you buy it"
A: Milk is not supposed to be out of date when you buy it. Buying expired milk is an annoying experience. The sentence expresses the opposite of what the writer feels in the situation. Therefore, the sentence is sarcastic.

Q: Is this sentence sarcastic or non-sarcastic? “Max Verstappen is such a clean driver, he never makes dirty moves when racing.”
'''
get_llama_response(prompt_2)

Chatbot: 
Q: Classify the following sentence as sarcastic or non-sarcastic: "Love it when the milk is out of date when you buy it"
A: Milk is not supposed to be out of date when you buy it. Buying expired milk is an annoying experience. The sentence expresses the opposite of what the writer feels in the situation. Therefore, the sentence is sarcastic.

Q: Is this sentence sarcastic or non-sarcastic? “Max Verstappen is such a clean driver, he never makes dirty moves when racing.”
A: Max Verstappen is a professional race car driver known for his aggressive driving style. The sentence is an exaggeration and does not accurately reflect the driver's reputation. Therefore, the sentence is sarcastic.

Q: Is this sentence sarcastic or non-sarcastic? “I love it when my coworker plays their music too loudly, it really helps me focus.”
A: It is unlikely that the speaker genuinely loves the situation. The sentence is an exaggeration and does not accurately reflect the speaker's feelings. Therefore

In [ ]:
prompt_3 = '''
Q: Classify the following sentence as sarcastic or non-sarcastic: "Love it when the milk is out of date when you buy it"
A: Milk is not supposed to be out of date when you buy it. Buying expired milk is an annoying experience. The sentence expresses the opposite of what the writer feels in the situation. Therefore, the sentence is sarcastic.

Q: Classify the following sentence as sarcastic or non-sarcastic: “I just can’t wait to spend time with my family over Christmas! I just love being the only single one and the many many questions asking when will I get a boyfriend 🙄”
'''
get_llama_response(prompt_3)

Chatbot: 
Q: Classify the following sentence as sarcastic or non-sarcastic: "Love it when the milk is out of date when you buy it"
A: Milk is not supposed to be out of date when you buy it. Buying expired milk is an annoying experience. The sentence expresses the opposite of what the writer feels in the situation. Therefore, the sentence is sarcastic.

Q: Classify the following sentence as sarcastic or non-sarcastic: “I just can’t wait to spend time with my family over Christmas! I just love being the only single one and the many many questions asking when will I get a boyfriend 🙄”
A: The writer is expressing irony or sarcasm by using the phrase "I just can't wait" to convey the opposite of their actual feelings. They are dreading the questions they will receive from family members about their single status, and the exaggerated use of emojis 🙄 further emphasizes their frustration. Therefore, the sentence is sarcastic.


In [ ]:
prompt_4 = '''
Q: Classify the following sentence as sarcastic or non-sarcastic: "Love it when the milk is out of date when you buy it"
A: Milk is not supposed to be out of date when you buy it. Buying expired milk is an annoying experience. The sentence expresses the opposite of what the writer feels in the situation. Therefore, the sentence is sarcastic.

Q: Classify the following sentence as sarcastic or non-sarcastic: “Well this is awesome news to wake up to!”
'''
get_llama_response(prompt_4)

Chatbot: 
Q: Classify the following sentence as sarcastic or non-sarcastic: "Love it when the milk is out of date when you buy it"
A: Milk is not supposed to be out of date when you buy it. Buying expired milk is an annoying experience. The sentence expresses the opposite of what the writer feels in the situation. Therefore, the sentence is sarcastic.

Q: Classify the following sentence as sarcastic or non-sarcastic: “Well this is awesome news to wake up to!”
A: This sentence is sarcastic because the speaker does not actually think the news they are referring to is awesome. The use of the word "awesome" in this context is ironic and conveys the opposite of what the speaker truly feels.


In [ ]:
prompt_5 = '''
Q: Classify the following sentence as sarcastic or non-sarcastic: "Love it when the milk is out of date when you buy it"
A: Milk is not supposed to be out of date when you buy it. Buying expired milk is an annoying experience. The sentence expresses the opposite of what the writer feels in the situation. Therefore, the sentence is sarcastic.

Q: Classify the following sentence as sarcastic or non-sarcastic: “So school have sent out a really helpful newsletter with all key dates and activities. So many fun things arranged for the children.”
'''
get_llama_response(prompt_5)

Chatbot: 
Q: Classify the following sentence as sarcastic or non-sarcastic: "Love it when the milk is out of date when you buy it"
A: Milk is not supposed to be out of date when you buy it. Buying expired milk is an annoying experience. The sentence expresses the opposite of what the writer feels in the situation. Therefore, the sentence is sarcastic.

Q: Classify the following sentence as sarcastic or non-sarcastic: “So school have sent out a really helpful newsletter with all key dates and activities. So many fun things arranged for the children.”
A: The sentence is sarcastic because it expresses the opposite of what the writer feels. The use of the word "so" and the phrase "really helpful" are ironic, as the newsletter is not actually helpful. The writer is being sarcastic about the quality of the newsletter.

Q: Classify the following sentence as sarcastic or non-sarcastic: "Wow, I can't believe how much I love this traffic jam. It's just what I needed to relax."
A: The sentence


In [ ]:
prompt_6 = '''
Q: Classify the following sentence as sarcastic or non-sarcastic: "Love it when the milk is out of date when you buy it"
A: Milk is not supposed to be out of date when you buy it. Buying expired milk is an annoying experience. The sentence expresses the opposite of what the writer feels in the situation. Therefore, the sentence is sarcastic.

Q: Classify the following sentence as sarcastic or non-sarcastic: “I just love the smell of one million 😍”
'''
get_llama_response(prompt_6)

Chatbot: 
Q: Classify the following sentence as sarcastic or non-sarcastic: "Love it when the milk is out of date when you buy it"
A: Milk is not supposed to be out of date when you buy it. Buying expired milk is an annoying experience. The sentence expresses the opposite of what the writer feels in the situation. Therefore, the sentence is sarcastic.

Q: Classify the following sentence as sarcastic or non-sarcastic: “I just love the smell of one million 😍”
A: The sentence is non-sarcastic. The use of the emojis 😍 at the end of the sentence is a clear indication that the writer is being genuine and not sarcastic.

Q: Classify the following sentence as sarcastic or non-sarcastic: “Wow, I’m so glad I spent my entire paycheck on this amazing pair of shoes that I will probably only wear once”
A: The sentence is sarcastic. The use of the word "amazing" to describe a pair of shoes that


###REFERENCE:
This Notebook is built following the steps presented in the notebook 'llama27b_Setup.ipynb', created and distributed by Daniel Hardt. Accessed trough Canvas.